# Multiple DataFrames

Не всегда бывает так что все данные помещаются в один набор, не редко полный набор разбивают на несколько частей (полная идентичность реляционным базам данных)

В таком случае у библиотеки Pandas существует специальный метод позволяющий нам объединять наборы данных в один по каким либо столбцам

Данный метод называется merge()

Метод можно вызвать как из класса pd так и из объекта класса DataFrame

`pd.merge(df1, df2)`

`df1.merge(df2)`

Если не указывать никакие дополнительные параметры то объединение будет произведенно по одноименным колонкам и одинаковым значениям в них

Существует различные виды merge:
- inner - внутреннее объединение
    - объединяет строки только с совпадающими значениями столбцов
- left - левое соединение
    - отображает все строки первого набора и только те строки значения которых совпали из второго набора. Все значения которые не совпали заполняет значением NaN
- rigth - правое соединение
    - отображает все строки второго набора и только те строки значения которых совпали из первого набора. Все значения которые не воспали заполняет значением NaN
- outer - внешнее соединение
    - отображает все строки как из первого так и из второго набора сопостовляет те строки значения столбцов которых совпали остальные значеия заполняет NaN

Правила идентичные правилам join в SQL

In [2]:
# Рассмотрим все на практических примерах
# для проверки будут использованы наборы данных расположенных по пути
# sales.csv; targets.csv

import pandas as pd

# загрузим данные
sales = pd.read_csv('./files/sales.csv')
targets = pd.read_csv('./files/targets.csv')

print(sales.head(), targets.head(), sep='\n\n')

      month  revenue
0   January      300
1  February      290
2     March      310
3     April      325
4       May      475

      month  target
0   January     310
1  February     270
2     March     300
3     April     350
4       May     475


In [3]:
# Объединим два набора используя метод merge()
# новый набор назовем sales_vs_targets

sales_vs_targets = pd.merge(sales, targets)

print(sales_vs_targets.head())
# судя по результату все получилось отлично
# pandas сам понял по какой колонке объединить два набора данных (одинаковые наименования и значения) и сформировал результат

      month  revenue  target
0   January      300     310
1  February      290     270
2     March      310     300
3     April      325     350
4       May      475     475


In [5]:
# давайте сравним показатели двух наборот и сохраним переменную в crushin_it
# переменная будет содержать только те строки где значение revenue больше target

crushin_it = sales_vs_targets[sales_vs_targets.revenue > sales_vs_targets.target].reset_index(drop=True)

print(crushin_it)

      month  revenue  target
0  February      290     270
1     March      310     300


In [6]:
# Добавим еще один набор данных men_women_sales.csv который будет находится по тому же пути что и остальные наборы
# загрузим данные в переменную men_women

men_women = pd.read_csv('./files/men_women_sales.csv')

print(men_women.head())

      month  men  women
0   January   30     35
1  February   29     35
2     March   31     29
3     April   32     28
4       May   47     50


In [8]:
# использовать merge можно друг за другом
# давайте попробуем объединить все наборы данных в один большой и назовем перменную all_data

all_data = pd.merge(sales, targets).merge(men_women).reset_index(drop=True)

print(all_data.head())
# как видим все получилось отлично

      month  revenue  target  men  women
0   January      300     310   30     35
1  February      290     270   29     35
2     March      310     300   31     29
3     April      325     350   32     28
4       May      475     475   47     50


In [9]:
# давайте попробуем получить из данных какую нибудь полезную информацию
# допустим получи месяца в которых revenue больше target и показатель women больше men
# сохраним данные в переменную results

results = all_data[(all_data.revenue > all_data.target) & (all_data.women > all_data.men)].reset_index(drop=True)

print(results.head())


      month  revenue  target  men  women
0  February      290     270   29     35


## Объединение наборов по определенным столбцам

Ранее мы производили объединение не выбирая определенные столбца и pandas по одинаковым названиям и значениям в столбцах предполагал как следует произвести объединение.

Но что если одинаковых наименований не будет?

В таком случае pandas не поймет что с чем соединять

Поэтому у метода merge() есть параметры которые мы можем использовать чтобы более конкретно указать что и как объединять

Параметры:
- left_on - колонка первого набора (первым набором считается тот что указан первым или от которого вызывается метод) (строковые данные)
- right_on - колонка второго набора (вторым набором считатется тот что указан вторым или который передается в метод) (строковые данные)
- how - тип объединения (см список выше) (строковые данные)
- suffixes - добавление идентификатора к одинаково названным столбцам (список строковых данных)

Для практической работы с параметрами создали дополнительные наборы orders_1.csv, products.csv

In [10]:
# Загрузим данные в перерменные
orders = pd.read_csv('./files/orders_1.csv')
products = pd.read_csv('./files/products.csv')

print(orders.head(), products.head(), sep='\n\n')

   id  product_id  customer_id  quantity   timestamp
0   1           3            2         1  2017-01-01
1   2           2            2         3  2017-01-01
2   3           1            3         1  2017-01-01
3   4           2            3         2  2016-02-01
4   5           3            3         3  2017-02-01

   id         description  price
0   1      thing-a-ma-jig      5
1   2  whatcha-ma-call-it     10
2   3          doo-hickey      7
3   4               gizmo      3


Мы видим в что в наборе присутствуют одинаково называнные колонки но то что подразумевают данные значения отличается

- колонка id в таблице orders - обозначает идентификатор заказа
- колонка id в таблице products - обозначает идентификатор продукта

Если мы попробуем их объединить все получиться но данные сопоставяться не верно что сильно повлияет на дальнейший анализ

Можно заметить что таблица orders содержит еще две колонки с добавочным словом id

- product_id
- customer_id

Из наименований можно предположить что они относятся к одно именным наборам данных. Получается что логичней и правильней следует объединять колонки id из таблицы products и колонку product_id из таблицы orders, в таком случае мы уже с большей уверенностью можем сказать что сопоставление пройдет более достоверно

По мимо использования параметров можно также просто переименовать столбец в таблице products (id на product_id) и произвести merge

In [13]:
# давайте проверим теорию
use_param = pd.merge(orders, products, left_on='product_id', right_on='id').reset_index(drop=True)
use_rename = pd.merge(orders, products.rename(columns={'id':'product_id'})).reset_index(drop=True)

print(use_param.head(), use_rename.head(), sep='\n\n')

# два полученных не отличаются сопоставление данных
# но отличаются наименованием столбцов в первом примере есть столбцы id_x и id_y хотя в начальных наборах их не было
# это произошло потому что они сохранились после объединения и чтобы не возникало ошибок pandas дал им suffixes-ы
# суффиксы мы можем указать и сами

add_suffixes = pd.merge(orders, products, left_on='product_id', right_on='id', suffixes=['_orders', '_products']).reset_index(drop=True)
print(add_suffixes.head())

   id_x  product_id  customer_id  quantity   timestamp  id_y  \
0     1           3            2         1  2017-01-01     3   
1     2           2            2         3  2017-01-01     2   
2     3           1            3         1  2017-01-01     1   
3     4           2            3         2  2016-02-01     2   
4     5           3            3         3  2017-02-01     3   

          description  price  
0          doo-hickey      7  
1  whatcha-ma-call-it     10  
2      thing-a-ma-jig      5  
3  whatcha-ma-call-it     10  
4          doo-hickey      7  

   id  product_id  customer_id  quantity   timestamp         description  \
0   1           3            2         1  2017-01-01          doo-hickey   
1   2           2            2         3  2017-01-01  whatcha-ma-call-it   
2   3           1            3         1  2017-01-01      thing-a-ma-jig   
3   4           2            3         2  2016-02-01  whatcha-ma-call-it   
4   5           3            3         3  2017-0

Для проверки различных типов соединений используем дополнительные наборы

store_a.csv, store_b.csv

In [14]:
# загрузим данные и проверим тип соединение outer
store_a = pd.read_csv('./files/store_a.csv')
store_b = pd.read_csv('./files/store_b.csv')

print(store_a.head(), store_b.head(), sep='\n\n')

          item  store_a_inventory
0       hammer                 12
1  screwdriver                 15
2        nails                200
3       screws                350
4          saw                  6

        item  store_b_inventory
0     hammer                  6
1      nails                250
2        saw                  6
3  duct tape                150
4   pvc pipe                 54


In [17]:
# объединим два набора типом соединения outer, left, right

outer_merge = pd.merge(store_a, store_b, how='outer').reset_index(drop=True)
left_merge = pd.merge(store_a, store_b, how='left').reset_index(drop=True)
right_merge = pd.merge(store_a, store_b, how='right').reset_index(drop=True)

print(outer_merge, left_merge, right_merge, sep='\n\n')
# внимательно посмотрите на полученные наборы и обратите внимание на строки в которых имеются значения NaN

             item  store_a_inventory  store_b_inventory
0       duct tape              150.0              150.0
1          hammer               12.0                6.0
2           nails              200.0              250.0
3        pvc pipe               54.0               54.0
4            rake                NaN               10.0
5             saw                6.0                6.0
6     screwdriver               15.0                NaN
7          screws              350.0                NaN
8          shovel                NaN               15.0
9   wooden dowels                NaN              192.0
10         wrench               12.0                NaN

          item  store_a_inventory  store_b_inventory
0       hammer                 12                6.0
1  screwdriver                 15                NaN
2        nails                200              250.0
3       screws                350                NaN
4          saw                  6                6.0
5    duct

## Concatenate DataFrame

Существует еще один вид соединения не построчный а по столбцам

Не редко бывает что одна большая таблица разбивается на 3 части в каждой из которых по 100 строк

Чтобы собрать их обратно используется метод concat()

Методу передается список dataframe-ов

Есть одно условие при котором это может сработать и оно заключается в том чтобы кол-во столбцов в каждом наборе данных должно быть одинаково как и их названия с последовательность. В противном случае новый набор получть очень "широким"

In [22]:
concat_df = pd.concat([store_a, store_b]).reset_index(drop=True)
concat_df_rename = pd.concat([store_a.rename(columns={'store_a_inventory':'inventory'}), store_b.rename(columns={'store_b_inventory':'inventory'})]).reset_index(drop=True)

print(concat_df, concat_df_rename, sep='\n\n')

             item  store_a_inventory  store_b_inventory
0          hammer               12.0                NaN
1     screwdriver               15.0                NaN
2           nails              200.0                NaN
3          screws              350.0                NaN
4             saw                6.0                NaN
5       duct tape              150.0                NaN
6          wrench               12.0                NaN
7        pvc pipe               54.0                NaN
8          hammer                NaN                6.0
9           nails                NaN              250.0
10            saw                NaN                6.0
11      duct tape                NaN              150.0
12       pvc pipe                NaN               54.0
13           rake                NaN               10.0
14         shovel                NaN               15.0
15  wooden dowels                NaN              192.0

             item  inventory
0          hammer 